# Quantitative Trading Strategy
#### By: Ruby Han

## Abstract
Forecasting market movement is a long-time attractive topic. This project attempts to create an algorithmic trading strategy to predict future asset returns and trigger buy or sell orders and benchmark strategy with the S&P 500 index. 

## Problem Objective 
- Create a quantitative trading strategy for any stocks
- No restriction on number of times entering or exiting the market, or long/short for period of strategy
- Benchmark with S&P 500 index
- Elucidate results and conclusion
- Provide future work and considerations given more time and resources

## Result Summary

## Navigation <a id = 0> </a>
- [Data Load](#1)
- [EDA](#2)
- [Feature Engineering](#3)
    - [MACD](#3.1)
    - [Bollinger](#3.2)
    - [RSI](#3.3)
    - [Normal Distribution](#3.4)
- [Model](#4)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from itertools import compress

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown as md

import yfinance as yf
import pandas_datareader as pdr

from feature_eng import MACD, bollinger_bands, RSI, momentum

import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, max_error

from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential

pd.set_option("display.max_rows", None, # display all rows
              "display.max_columns", None, # display all columns
              "display.max_colwidth", None, # expand column width
              "display.html.use_mathjax", False) # disable Latex style mathjax rendering

## Data Load <a id = 1> </a> 
[Back to Top](#0)

- S&P 500 index data 

##### Terminology
- `Open` : Stock price at market open (USD)
- `High` : Highest price reached per day (USD)
- `Low` : Lowest price reached per day (USD)
- `Close` : Stock price at market close (USD)
- `Adj Close` : Adjusted stock price at market close (USD)
- `Volume` : Number of shares traded per day

In [ ]:
# Obtain stock data from Yahoo Finance
start = '2010-01-01'
end = '2022-01-01'
# data = yf.download('AAPL', start=start, end=end)

tickers = [
    '^GSPC' # sp500
    ,'XOM' # exxon
    ,'CVX' # chevron
    ,'AAPL' # apple
    ,'TSLA' # tesla
    ,'MSFT' # microsoft
          ]

for ticker in tickers:
    data = pdr.get_data_yahoo(ticker, '2000')
    data.to_csv(f'data/raw_stocks/{ticker}.csv')

# data = yf.download('AAPL', start=start, end=end)

In [ ]:
apple_raw_df = pd.read_csv('data/raw_stocks/AAPL.csv')

## EDA <a id = 2> </a> 
[Back to Top](#0)

In [ ]:
apple_raw_df.head()

In [ ]:
apple_raw_df.shape

In [ ]:
apple_raw_df.describe()

In [ ]:
apple_raw_df.dtypes

In [ ]:
# Check for null values
# If exist, use last good value, mean, zero or drop observation
apple_raw_df.isna().sum()

## Feature Engineering <a id = 3> </a> 
[Back to Top](#0)

- Indicators are tools that help traders/investors make buying/selling stocks decisions
- Technical indicators (features in our case):
    - Price
    - Volume
- The following features will be created:
    - Bollinger Bands
    - RSI
    - MACD
    - Moving Average
    - Momentum
    - Change
    - Volatility
    - Return
- Target variable: `Return`
- All other features will serve as predictors

In [ ]:
!rm -rf data/raw_stocks/.ipynb_checkpoints
files = os.listdir('data/raw_stocks')
stocks = {}

for file in files:
    name = file.split('.')[0]
    stocks[name] = pd.read_csv(f'data/raw_stocks/{file}') 
    
    stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
    stocks[name].set_index('Date', inplace=True)
    
    # Bollinger Bands
    stocks[name]['upper_boll_band'], stocks[name]['lower_boll_band'] = bollinger_bands(stocks[name])
    
    # MACD
    stocks[name]['macd'], stocks[name]['signal'] = MACD(stocks[name])
    
    # RSI
    stocks[name]['rsi'] = RSI(stocks[name])
    
    # 7d MA
    stocks[name]['ma7'] = stocks[name].Close.rolling(window=7).mean()
    
    # 21d MA
    stocks[name]['ma21'] = stocks[name].Close.rolling(window=21).mean()
    
    # Momentum
    stocks[name]['momentum'] = momentum(stocks[name].Close, 3)
    
    # Difference between current and previous
    stocks[name]['change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
    
    # Volatility
    stocks[name]['volatility'] = stocks[name].Close.ewm(21).std()
    
    # Return
    stocks[name]['return'] = round(stocks[name]['Close']/stocks[name]['Open'] - 1, 4)

    stocks[name].to_csv(f'data/stocks/{name}.csv')

In [ ]:
files

In [ ]:
apple_df = stocks['AAPL']

#### Correlation Plot
- `change` and `rsi` correlate with `return`

In [ ]:
corr = apple_df.corr()

# Mask upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Plot correlation matrix
plt.figure(figsize=(16, 5))
heatmap = sns.heatmap(corr, mask=mask, annot=True, linewidths=0.5, 
                      vmin=-0.75, vmax=0.75, cmap="RdBu_r")
heatmap.set_title('Correlation Between Features');

In [ ]:
corr[['return']].sort_values(by='return', ascending=False)

## $\underline{\text{Plots}}$

### MACD - Moving Average Convergence Divergence  <a id = 3.1> </a> 
[Back to Top](#0)
- Momentum indicator showing relationship between two moving averages
- Logic is that momentum has more impact on short moving average and we subtract short_ma from long_ma
- Difference is sometimes positive or negative, hence the name MACD (moving average converge/diverge oscillator)
    - Oscillator is the difference between the two MAs
    - When it is positive, we long and vice versa
- If short_ma > long_ma, then long and hold as stock is on the rise and will keep going up for some time
- If short_ma < long_ma, then clear positions

In [ ]:
def macd(stock):
    plt.figure(figsize=(16,5))
    plt.plot(stock.macd, label='short_ma', color = '#b278ff')
    plt.plot(stock.signal, label='long_ma', color='#ffa74a')
    plt.axhline(0, color='#557692')
    plt.legend(frameon=True, loc=0, ncol=1, fontsize=10, borderpad=.6)
    plt.title('MACD', fontsize=15)
    plt.ylabel('Strength', fontsize=12)
    plt.show()
    
    plt.figure(figsize=(16,5))
    (stock.macd - stock.signal).plot(kind='bar',color='r')
    plt.grid(True)
    plt.xticks([])
    plt.xlabel('')
    plt.title('MACD Oscillator', fontsize=15)
    plt.show()

In [ ]:
macd(apple_df.loc['2020':'2020'])

### Bollinger Bands  <a id = 3.2> </a> 
[Back to Top](#0)
- Price almost never leaves Bollinger Bands space
    - Price is fluctuating between three standard deviations
- Good indicator as buying/selling signal

In [ ]:
def bollinger_bands_plot(stock, std=3):    
    plt.figure(figsize=(16,5))
    plt.style.use('seaborn-whitegrid')
    plt.plot(stock.index, stock.Close, color='#3388cf', label='Price')
    plt.plot(stock.index, stock.ma21, color='#ad6eff', label='Moving Average (21 days)')
    plt.plot(stock.index, stock.ma7, color='#ff6e9d', label='Moving Average (7 days)')
    plt.plot(stock.index, stock.upper_boll_band, color='#ffbd74', alpha=0.3)
    plt.plot(stock.index, stock.lower_boll_band, color='#ffa33f', alpha=0.3)
    plt.fill_between(stock.index, stock.upper_boll_band, stock.lower_boll_band, color='#ffa33f', alpha=0.1, label=f'Bollinger Band ({std} STD)')
    plt.legend(frameon=True, loc=0, ncol=1, fontsize=10, borderpad=.6)
    plt.title('Bollinger Bands', fontsize=15)
    plt.ylabel('Price', fontsize=12)
    plt.xlim([stock.index.min(), stock.index.max()])
    plt.show()

In [ ]:
bollinger_bands_plot(apple_df.loc['2020':'2020'])

### RSI - Relative Strength Index <a id = 3.3> </a> 
[Back to Top](#0)
- A momentum indicator that can tell if stock is overbought or oversold
- Ranges from 0 to 100
- When index approaches 30, signal to buy
- When index approaches 70, signal to sell

In [ ]:
def rsi(stock):
    plt.figure(figsize=(16,5)) 
    plt.plot(stock.index, stock.rsi, color='#ad6eff')
    plt.xlim([stock.index.min(), stock.index.max()])
    plt.axhline(30, color='#f9989c')
    plt.axhline(70, color='#60e8ad')
    plt.title('RSI', fontsize=15)
    plt.ylabel('%', fontsize=12)
    plt.ylim([0, 100])
    plt.show()

In [ ]:
rsi(apple_df.loc['2020':'2020'])

### Normal Distribution <a id = 3.4> </a> 
[Back to Top](#0)
- ML algorithms require normal distribution of data to work well
- Target var `return` mostly normal

In [ ]:
apple_df.head()

In [ ]:
z = lambda x: (x - x.mean()) / x.std()

plt.hist(z(apple_df['return']), bins=30)
plt.title('Apple Return Normal Distribution', fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
sm.qqplot(apple_df['return'], line='s', scale=1)
plt.title('QQPlot', fontsize=15);

## Model <a id = 4> </a> 
[Back to Top](#0)
- Compare different machine learning models

### Data Preparation
- Scale data to [0-1] using MinMaxScaler 
    - Achieve greater precision by scaling down from huge values
    - Reduce computational cost
    - Reduce memory consumption

In [ ]:
# Scaling
def scale(stock):
    scaler = MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(stock), columns=apple_df.columns)

In [ ]:
dataset = apple_df.dropna()

X = scale(dataset).drop(['return'], axis=1)
y = dataset['return']

In [ ]:
# final_features = X.columns.drop(['32FC0744BO', '32FC0749_O']).to_list()

In [ ]:
estimator = LinearRegression(n_jobs=-1)
selector = RFE(estimator, step=1)
selector = selector.fit(X, y)

features = X.columns.to_list()

final_features = list(compress(features, selector.support_))
selector.support_
selector.ranking_

final_features

In [ ]:
# Time series train and test data split at regular time intervals
def data_split(features, n_split=10):
    timesplit= TimeSeriesSplit(n_splits=n_split)
    for train_index, test_index in timesplit.split(features):
        X_train, X_test = features[:len(train_index)], features[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = y[:len(train_index)].values.ravel(), y[len(train_index): (len(train_index)+len(test_index))].values.ravel()
        return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = data_split(X[final_features])

In [ ]:
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mse', optimizer='adam', metrics=['mae'])
lstm.summary()

In [ ]:
history=lstm.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1, shuffle=False)

In [ ]:
y_pred= lstm.predict(X_test)

In [ ]:
#Predicted vs True Adj Close Value – LSTM
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('LSTM Model')
plt.xlabel('Time Scale')
plt.ylabel('Scaled Price')
plt.legend()
plt.show()

# MACD

In [ ]:
start = '2018-01-05'
end = '2019-01-05'
ticker = 'XOM' # '^GSPC'
data = yf.download(ticker, start=start, end=end)
short_ma = 10
long_ma = 21

In [ ]:
# Simple moving average
data['short_ma'] = data.Close.rolling(window=short_ma).mean()   
data['long_ma'] = data.Close.rolling(window=long_ma).mean()

In [ ]:
data.tail()

In [ ]:
data['short_ma'] = data.Close.ewm(short_ma).mean()   
data['long_ma'] = data.Close.ewm(long_ma).mean()

In [ ]:
data.tail()

In [ ]:
# If short_ma > long_ma, then long and hold
# If short_ma < long_ma, then clear positions

# Logic is that momentum has more impact on short moving average and we subtract short_ma from long_ma
# Difference is sometimes positive or negative, hence the name MACD (moving average converge/diverge oscillator)

# Generate trade signal as 1 when short_ma > long_ma (hold), else -1 (clear position)
data['signal'] = np.where(data['short_ma'] > data['long_ma'], 1, -1)

# Oscillator is the difference between the two MAs
# When it is positive, we long and vice versa
data['oscillator'] = data['short_ma'] - data['long_ma']

In [ ]:
# Plotting the backtesting result
# The first plot is the actual close price with long/short positions
fig=plt.figure()
ax=fig.add_subplot(111)

data.Close.plot(label=ticker)
ax.plot(data.loc[data['signal']==1].index, data['Close'][data['signal']==1], label='Long', lw=0,marker='^', c='g')
ax.plot(data.loc[data['signal']==-1].index, data['Close'][data['signal']==-1], label='Short', lw=0,marker='v', c='r')

plt.legend(loc='best')
plt.grid(True)
plt.title('Positions')

plt.show()

# The second plot (bar chart) is long/short moving average with oscillator
fig=plt.figure()
cx=fig.add_subplot(211)

# Oscillator is the difference between the two MAs
# When it is positive, we long and vice versa
data['oscillator'].plot(kind='bar',color='r')

plt.legend(loc='best')
plt.grid(True)
plt.xticks([])
plt.xlabel('')
plt.title('MACD Oscillator')

bx=fig.add_subplot(212)

data['short_ma'].plot(label='short_ma')
data['long_ma'].plot(label='long_ma',linestyle=':')

plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# Backtesting
# Initial capital of $10k to calculate P&L
# 100 shares to buy every position

capital0 = 10000
positions = 100

# Cumulative summation column is created to check holding of the position
data['cumsum']=data['signal'].cumsum()

portfolio = pd.DataFrame()
portfolio['holdings'] = data['cumsum']*data.Close*positions
portfolio['cash'] = capital0 - (data['signal']*data.Close*positions).cumsum()
portfolio['total_asset'] = portfolio['holdings'] + portfolio['cash']
portfolio['return'] = portfolio['total_asset'].pct_change()
portfolio['signal'] = data['signal'].copy()
portfolio['date'] = data.index
portfolio.set_index('date',inplace=True)

In [ ]:
# Plotting the asset value change of the portfolio
        
fig=plt.figure()
bx=fig.add_subplot(111)

portfolio['total_asset'].plot(label='Total Asset')

# Long/short position markers related to the portfolio
# The same mechanism as the previous one
# Replace close price with total asset value
bx.plot(portfolio['signal'].loc[portfolio['signal']==1].index,portfolio['total_asset'][portfolio['signal']==1],lw=0,marker='^',c='g',label='long')
bx.plot(portfolio['signal'].loc[portfolio['signal']<0].index,portfolio['total_asset'][portfolio['signal']<0],lw=0,marker='v',c='r',label='short')

plt.legend(loc='best')
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Asset Value')
plt.title('Total Asset')
plt.show()

In [ ]:
# Metrics
def stats(portfolio=portfolio, trading_signals=data,stdate=start,eddate=end,capital0=10000):

    stats=pd.DataFrame([0])

    #get the min and max of return
    maximum=np.max(portfolio['return'])
    minimum=np.min(portfolio['return'])    

    #growth_rate denotes the average growth rate of portfolio 
    #use geometric average instead of arithmetic average for percentage growth
    growth_rate=(float(portfolio['total_asset'].iloc[-1]/capital0))**(1/len(trading_signals))-1

    #calculating the standard deviation
    std=float(np.sqrt((((portfolio['return']-growth_rate)**2).sum())/len(trading_signals)))

    #use S&P500 as benchmark
    benchmark=yf.download('^GSPC',start=stdate,end=eddate)

    #return of benchmark
    return_of_benchmark=float(benchmark['Close'].iloc[-1]/benchmark['Open'].iloc[0]-1)

    #rate_of_benchmark denotes the average growth rate of benchmark 
    #use geometric average instead of arithmetic average for percentage growth
    rate_of_benchmark=(return_of_benchmark+1)**(1/len(trading_signals))-1

    del benchmark

    #backtesting stats
    #CAGR stands for cumulated average growth rate
    stats['CAGR']=stats['portfolio return']=float(0)
    stats['CAGR'][0]=growth_rate
    stats['portfolio return'][0]=portfolio['total_asset'].iloc[-1]/capital0-1
    stats['benchmark return']=return_of_benchmark
    stats['sharpe ratio']=(growth_rate-rate_of_benchmark)/std

    #note that i use stop loss limit to limit the numbers of longs
    #and when clearing positions, we clear all the positions at once
    #so every long is always one, and short cannot be larger than the stop loss limit
    stats['numbers of longs']=trading_signals['signal'].loc[trading_signals['signal']==1].count()
    stats['numbers of shorts']=trading_signals['signal'].loc[trading_signals['signal']<0].count()
    stats['numbers of trades']=stats['numbers of shorts']+stats['numbers of longs']  

    #to get the total length of trades
    #given that cumsum indicates the holding of positions
    #we can get all the possible outcomes when cumsum doesnt equal zero
    #then we count how many non-zero positions there are
    #we get the estimation of total length of trades
    stats['total length of trades']=trading_signals['signal'].loc[trading_signals['cumsum']!=0].count()
    stats['average length of trades']=stats['total length of trades']/stats['numbers of trades']
    stats['profit per trade']=float(0)
    stats['profit per trade'].iloc[0]=(portfolio['total_asset'].iloc[-1]-capital0)/stats['numbers of trades'].iloc[0]

    del stats[0]
    print(stats)

In [ ]:
stats()

In [ ]:
df = data.copy()
df['Log Returns'] = np.log(data['Close']/data['Close'].shift(1))
df.dropna(inplace=True)
df['Strategy Log Returns'] = df['Log Returns']*df['signal'].shift(1)
df.dropna(inplace=True)

In [ ]:
def MA_Strategy(stock=ticker,start=start,end=end,MAF=short_ma,MAS=long_ma):
    data = yf.download(stock, start=start, end=end)
    data['Log Returns'] = np.log(data['Close']/data['Close'].shift(1))
    data.dropna(inplace=True)
    data['MASlow'] = data['Close'].rolling(MAS).mean()
    data['MAFast'] = data['Close'].rolling(MAF).mean()
    data.dropna(inplace=True)
    data['Signal'] = np.where(data['MAFast']>data['MASlow'],1,-1)
    data.dropna(inplace=True)
    data['Strategy Log Returns'] = data['Log Returns'] * data['Signal'].shift(1)
    data.dropna(inplace=True)
    
    # We show the results:
    
    data[['Close','MAFast','MASlow','Signal']].plot(
    secondary_y='Signal',
    figsize=(15,9),
    title=f'Close Price for {ticker}, Short/Long Moving Averages ({MAF}d and {MAS}d) and Short/Long Signal')\
    .get_legend().set_bbox_to_anchor((1.17, 0.85))

    plt.figure(figsize = (15,9))
    (100*(np.exp(data['Strategy Log Returns'].cumsum())-1)).plot(label = 'Cumulative Returns (Strategy)')
    (100*(np.exp(data['Log Returns'].cumsum())-1)).plot(label = 'Cumulative Returns (Buy and Hold)')
    plt.title('Cumulative Returns (%): {}'.format(stock))
    plt.legend()
    plt.show()
    
    print('\n')
    print('Total Returns:', stock)
    print(35*'===')
    print('Market Total Return:',((np.exp(data[['Log Returns', 'Strategy Log Returns']].sum()) -1)*100).round(3).iloc[0],'%')
    print('Strategy Total Return:',((np.exp(data[['Log Returns', 'Strategy Log Returns']].sum()) -1)*100).round(3).iloc[1],'%')
    print('\n')
    print('Annualized Volatility:', stock)
    print(35*'===')
    print('Market Volatility:',(data[['Log Returns', 'Strategy Log Returns']].std()*252**0.5).round(5).iloc[0],'%')
    print('Strategy Volatility:',(data[['Log Returns', 'Strategy Log Returns']].std()*252**0.5).round(5).iloc[1],'%')

In [ ]:
MA_Strategy()